In [1]:
#SETTING PATH 
previousResults="/sharedFolder/Results/Hotspot/Revision1/FULL/"
results="/sharedFolder/Results/Hotspot/Revision1/FULL/"
dir.create(results,recursive=TRUE)
riderPath="/home/Riderv2/"


Warning message in dir.create(results, recursive = TRUE):
“'/sharedFolder/Results/Hotspot/Revision1/FULL' already exists”


In [1]:
#LOAD USEFULL FUNCTIONS 
noExt=function(x){
return(tools::file_path_sans_ext(x))
}
bnoExt=function(x){
return(basename(tools::file_path_sans_ext(x)))
}
fNumber=function(x){
sapply(x,FUN=function(x){
strsplit(x,"__")[[1]][2]
})
}
rRider=function(x){
sapply(x,FUN=function(x){
strsplit(x,"-RIDER")[[1]][1]
})
}
# centering with 'scale()'
center_scale <- function(x) {
    scale(x, scale = FALSE)
}

In [2]:
#LOAD LIBRARIES 
library("argparser");
library(gtools);
library("GenomicRanges");
library("ggVennDiagram");
options(warn=-1)
library(GenomicRanges)

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin,
    pmin.int, rank, rbind, rownames, sapply, setdiff, sort, table,
    tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    I, expand.grid, unname


Loading required package: IRanges

Loading required package: GenomeInfoDb



In [31]:
library(beeswarm)

jamesonFirst = 7
message("Processing threshold: ", jamesonFirst)

data_file <- paste("/sharedFolder/Data/7_newCRIPRS_Res/7/heatmap_Combo/count_hgtgtsANNOTATED.csv", sep = "")
data <- read.csv(data_file, header = TRUE, row.names = 1, sep = ",")
data <- data + 1  # Pseudocount

# Calcolo Fold Change
foldChange_IDE_DMSO <- data[,"MEC.1_Idelalisib_DMSO"] / data[,"AID.WT_DMSO"]
foldChange_IDE_Taze <- data[,"MEC.1_Idelalisib_Tazemetostat"] / data[,"AID.WT_DMSO"]
foldChange_IDE_Vale <- data[,"MEC.1_Idelalisib_Valemetostat"] / data[,"AID.WT_DMSO"]
foldChange_AID_WT_Taze <- data[,"AID.WT_Tazemetostat"] / data[,"AID.WT_DMSO"]
foldChange_AID_WT_Vale <- data[,"AID.WT_Valemetostat"] / data[,"AID.WT_DMSO"]
dmso_WT <- data[,"AID.WT_DMSO"] / data[,"AID.WT_DMSO"]

foldChange_table <- data.frame(
  DMSO = dmso_WT,
  Tazemetostat = foldChange_AID_WT_Taze,
  Valemetostat = foldChange_AID_WT_Vale,
  Idelalisib = foldChange_IDE_DMSO,
  `Idelalisib+Tazemetostat` = foldChange_IDE_Taze,
  `Idelalisib+Valemetostat` = foldChange_IDE_Vale
)

# PNG output
png(
  filename = paste("/sharedFolder/Data/7_newCRIPRS_Res/7/heatmap_Combo/count_boxPlot.png", sep = ""),
  width = 2000,
  height = 1500,
  res = 300,
  type = "cairo"
)

# Boxplot
boxplot(foldChange_table, xaxt = "n", main = paste("Fold Change - threshold", jamesonFirst),
        xlab = "", ylab = "Fold Change", outline = FALSE, col = "gray90", border = "gray40",ylim=c(-1,18))

# Linea di riferimento a y = 1
abline(h = 1, col = "black", lty = 2, lwd = 1)

# Beeswarm o punto singolo
for (i in 1:ncol(foldChange_table)) {
  color <- if (grepl("Idelalisib", colnames(foldChange_table)[i], ignore.case = TRUE)) "red" else "blue"
  
  if (colnames(foldChange_table)[i] == "DMSO") {
    points(i, 1, col = color, pch = 19, cex = 1)  # Un solo punto per DMSO
  } else {
    beeswarm(foldChange_table[[i]], add = TRUE, at = i, col = color, pch = 20, vertical = TRUE)
  }
}

# Etichette asse X
labels <- colnames(foldChange_table)
axis(1, at = 1:length(labels), labels = FALSE)
text(x = 1:length(labels), y = par("usr")[3] - 0.3, srt = 45, adj = 1,
     labels = labels, xpd = TRUE, cex = 0.7)

dev.off()

  
  message("Processing threshold: ", jamesonFirst)
  
data_file <- paste("/sharedFolder/Data/7_newCRIPRS_Res/7/heatmap_Combo/count_hgtgtsANNOTATED.csv", sep = "")
  a <- read.csv(data_file, header = TRUE, row.names = 1, sep = ",")
  
  conditions <- colnames(a)
  reference_condition <- a$AID.WT_DMSO
  
  # Inizializza vettore per p-value
  p_values <- numeric(length(conditions))
  names(p_values) <- conditions
  
  # Test Wilcoxon vs AID.WT_DMSO
  for (i in 1:length(conditions)) {
    cond <- conditions[i]
    if (cond != "AID.WT_DMSO") {
      p_values[cond] <- wilcox.test(a[[cond]], reference_condition)$p.value
    } else {
      p_values[cond] <- NA  # Reference vs se stessa
    }
  }

  # Confronti Idelalisib combinato vs Idelalisib DMSO
  p_values[["Idelalisib_Taze_vs_Idelalisib_DMSO"]] <- wilcox.test(a$MEC.1_Idelalisib_Tazemetostat, a$MEC.1_Idelalisib_DMSO)$p.value
  p_values[["Idelalisib_Vale_vs_Idelalisib_DMSO"]] <- wilcox.test(a$MEC.1_Idelalisib_Valemetostat, a$MEC.1_Idelalisib_DMSO)$p.value

  # Confronti Idelalisib combinato vs WT combinato
  p_values[["Idelalisib_Taze_vs_WT_Taze"]] <- wilcox.test(a$MEC.1_Idelalisib_Tazemetostat, a$AID.WT_Tazemetostat)$p.value
  p_values[["Idelalisib_Vale_vs_WT_Vale"]] <- wilcox.test(a$MEC.1_Idelalisib_Valemetostat, a$AID.WT_Valemetostat)$p.value

  # Stampa
  cat("\nP-values for threshold", jamesonFirst, ":\n")
  for (n in names(p_values)) {
    cat(n, ":", p_values[n], "\n")
  }

  # Salvataggio su file
  pval_out <- paste("/sharedFolder/Data/7_newCRIPRS_Res/7/heatmap_Combo/pvalCount.csv", sep = "")
  write.csv(as.data.frame(p_values), file = pval_out, quote = FALSE)



Processing threshold: 7



png 
  2

Processing threshold: 7




P-values for threshold 7 :
AID.WT_DMSO : NA 
AID.WT_Tazemetostat : 0.8896115 
AID.WT_Valemetostat : 0.7286194 
MEC.1_AID.KO_DMSO23 : 0.0439745 
MEC.1_AID.KO_Tazemetostat57 : 0.1049232 
MEC.1_AID.KO_Valemetostat1011 : 0.03854531 
MEC.1_Idelalisib_DMSO : 0.1043619 
MEC.1_Idelalisib_Tazemetostat : 0.0116915 
MEC.1_Idelalisib_Valemetostat : 0.01853335 
Idelalisib_Taze_vs_Idelalisib_DMSO : 0.4376508 
Idelalisib_Vale_vs_Idelalisib_DMSO : 0.427111 
Idelalisib_Taze_vs_WT_Taze : 0.003698333 
Idelalisib_Vale_vs_WT_Vale : 0.009940665 


In [13]:
library(beeswarm)

jamesonFirst = 7
message("Processing threshold: ", jamesonFirst)

data_file <- paste("/sharedFolder/Data/7_newCRIPRS_Res/7/heatmap_Combo/rpm_hgtgtsANNOTATED.csv", sep = "")
data <- read.csv(data_file, header = TRUE, row.names = 1, sep = ",")
data <- data + 1  # Pseudocount

# Calcolo Fold Change
foldChange_IDE_DMSO <- data[,"MEC.1_Idelalisib_DMSO"] / data[,"AID.WT_DMSO"]
foldChange_IDE_Taze <- data[,"MEC.1_Idelalisib_Tazemetostat"] / data[,"AID.WT_DMSO"]
foldChange_IDE_Vale <- data[,"MEC.1_Idelalisib_Valemetostat"] / data[,"AID.WT_DMSO"]
foldChange_AID_WT_Taze <- data[,"AID.WT_Tazemetostat"] / data[,"AID.WT_DMSO"]
foldChange_AID_WT_Vale <- data[,"AID.WT_Valemetostat"] / data[,"AID.WT_DMSO"]
dmso_WT <- data[,"AID.WT_DMSO"] / data[,"AID.WT_DMSO"]

foldChange_table <- data.frame(
  DMSO = dmso_WT,
  Tazemetostat = foldChange_AID_WT_Taze,
  Valemetostat = foldChange_AID_WT_Vale,
  Idelalisib = foldChange_IDE_DMSO,
  `Idelalisib+Tazemetostat` = foldChange_IDE_Taze,
  `Idelalisib+Valemetostat` = foldChange_IDE_Vale
)

# PNG output
png(
  filename = paste("/sharedFolder/Data/7_newCRIPRS_Res/7/heatmap_Combo/rpm_boxPlot.png", sep = ""),
  width = 2000,
  height = 1500,
  res = 300,
  type = "cairo"
)

# Boxplot
boxplot(foldChange_table, xaxt = "n", main = paste("Fold Change - threshold", jamesonFirst),
        xlab = "", ylab = "Fold Change", outline = FALSE, col = "gray90", border = "gray40")

# Linea di riferimento a y = 1
abline(h = 1, col = "black", lty = 2, lwd = 1)

# Beeswarm o punto singolo
for (i in 1:ncol(foldChange_table)) {
  color <- if (grepl("Idelalisib", colnames(foldChange_table)[i], ignore.case = TRUE)) "red" else "blue"
  
  if (colnames(foldChange_table)[i] == "DMSO") {
    points(i, 1, col = color, pch = 19, cex = 1)  # Un solo punto per DMSO
  } else {
    beeswarm(foldChange_table[[i]], add = TRUE, at = i, col = color, pch = 20, vertical = TRUE)
  }
}

# Etichette asse X
labels <- colnames(foldChange_table)
axis(1, at = 1:length(labels), labels = FALSE)
text(x = 1:length(labels), y = par("usr")[3] - 0.3, srt = 45, adj = 1,
     labels = labels, xpd = TRUE, cex = 0.7)

dev.off()

  
  message("Processing threshold: ", jamesonFirst)
  
data_file <- paste("/sharedFolder/Data/7_newCRIPRS_Res/7/heatmap_Combo/rpm_hgtgtsANNOTATED.csv", sep = "")
  a <- read.csv(data_file, header = TRUE, row.names = 1, sep = ",")
  
  conditions <- colnames(a)
  reference_condition <- a$AID.WT_DMSO
  
  # Inizializza vettore per p-value
  p_values <- numeric(length(conditions))
  names(p_values) <- conditions
  
  # Test Wilcoxon vs AID.WT_DMSO
  for (i in 1:length(conditions)) {
    cond <- conditions[i]
    if (cond != "AID.WT_DMSO") {
      p_values[cond] <- wilcox.test(a[[cond]], reference_condition)$p.value
    } else {
      p_values[cond] <- NA  # Reference vs se stessa
    }
  }

  # Confronti Idelalisib combinato vs Idelalisib DMSO
  p_values[["Idelalisib_Taze_vs_Idelalisib_DMSO"]] <- wilcox.test(a$MEC.1_Idelalisib_Tazemetostat, a$MEC.1_Idelalisib_DMSO)$p.value
  p_values[["Idelalisib_Vale_vs_Idelalisib_DMSO"]] <- wilcox.test(a$MEC.1_Idelalisib_Valemetostat, a$MEC.1_Idelalisib_DMSO)$p.value

  # Confronti Idelalisib combinato vs WT combinato
  p_values[["Idelalisib_Taze_vs_WT_Taze"]] <- wilcox.test(a$MEC.1_Idelalisib_Tazemetostat, a$AID.WT_Tazemetostat)$p.value
  p_values[["Idelalisib_Vale_vs_WT_Vale"]] <- wilcox.test(a$MEC.1_Idelalisib_Valemetostat, a$AID.WT_Valemetostat)$p.value

  # Stampa
  cat("\nP-values for threshold", jamesonFirst, ":\n")
  for (n in names(p_values)) {
    cat(n, ":", p_values[n], "\n")
  }

  # Salvataggio su file
  pval_out <- paste("/sharedFolder/Data/7_newCRIPRS_Res/7/heatmap_Combo/pvalrpm.csv", sep = "")
  write.csv(as.data.frame(p_values), file = pval_out, quote = FALSE)



Processing threshold: 7



png 
  2

Processing threshold: 7




P-values for threshold 7 :
AID.WT_DMSO : NA 
AID.WT_Tazemetostat : 1 
AID.WT_Valemetostat : 0.5332601 
MEC.1_AID.KO_DMSO23 : 0.07488871 
MEC.1_AID.KO_Tazemetostat57 : 0.187111 
MEC.1_AID.KO_Valemetostat1011 : 0.08062963 
MEC.1_Idelalisib_DMSO : 0.3696238 
MEC.1_Idelalisib_Tazemetostat : 0.3008692 
MEC.1_Idelalisib_Valemetostat : 0.2838609 
Idelalisib_Taze_vs_Idelalisib_DMSO : 0.8362004 
Idelalisib_Vale_vs_Idelalisib_DMSO : 0.9175939 
Idelalisib_Taze_vs_WT_Taze : 0.02972276 
Idelalisib_Vale_vs_WT_Vale : 0.06214911 


In [36]:
write.table(test,"/sharedFolder/Data/7_newCRIPRS_Res/7/heatmap_Combo/fc.csv",col.names=NA,sep=",",row.names=FALSE)

ERROR: Error in write.table(test, "/sharedFolder/Data/7_newCRIPRS_Res/7/heatmap_Combo/fc.csv", : 'col.names = NA' makes no sense when 'row.names = FALSE'
